In [4]:
# Dependencies
import os, cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, preprocessing
from sklearn.model_selection import train_test_split

In [38]:
# Read training and testing images into arrays
realTrain = []
fakeTrain = []
realTest = []
fakeTest = []

for filename in os.listdir("./data/train/REAL"):
    imgPath = os.path.join("./data/train/REAL", filename)
    if os.path.isfile(imgPath):
        img = cv2.imread(imgPath)
        if img is not None:
            realTrain.append(img)

for filename in os.listdir("./data/train/FAKE"):
    imgPath = os.path.join("./data/train/FAKE", filename)
    if os.path.isfile(imgPath):
        img = cv2.imread(imgPath)
        if img is not None:
            fakeTrain.append(img)

for filename in os.listdir("./data/test/REAL"):
    imgPath = os.path.join("./data/test/REAL", filename)
    if os.path.isfile(imgPath):
        img = cv2.imread(imgPath)
        if img is not None:
            realTest.append(img)

for filename in os.listdir("./data/test/FAKE"):
    imgPath = os.path.join("./data/test/FAKE", filename)
    if os.path.isfile(imgPath):
        img = cv2.imread(imgPath)
        if img is not None:
            fakeTest.append(img)

realTrain = np.array(realTrain)
fakeTrain = np.array(fakeTrain)
realTest = np.array(realTest)
fakeTest = np.array(fakeTest)

xTrain = np.concatenate((realTrain, fakeTrain), axis=0)
yTrain = np.concatenate((np.ones(len(realTrain)), np.zeros(len(fakeTrain))))
xTest = np.concatenate((realTest, fakeTest), axis=0)
yTest = np.concatenate((np.ones(len(realTest)), np.zeros(len(fakeTest))))

print(f"""
realTrain legnth: {len(realTrain)}
fakeTrain length: {len(fakeTrain)}
realTest length: {len(realTest)}
fakeTest length: {len(fakeTest)}
xTrain length: {len(xTrain)}
yTrain length: {len(yTrain)}
xTest length: {len(xTest)}
yTest length: {len(yTest)}""")


realTrain legnth: 50000
fakeTrain length: 50000
realTest length: 10000
fakeTest length: 10000
xTrain length: 100000
yTrain length: 100000
xTest length: 20000
yTest length: 20000


In [39]:
# Train and test the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(xTrain, yTrain, epochs=30, validation_split=0.2)

testLoss, testAccuracy = model.evaluate(xTest, yTest)
trainLoss, trainAccuracy = model.evaluate(xTrain, yTrain)
print(f"Test Loss: {testLoss}\nTest Accuracy: {testAccuracy}")
print(f"Train Loss: {trainLoss}\nTrain Accuracy: {trainAccuracy}")

Epoch 1/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.7790 - loss: 0.6500 - val_accuracy: 0.8375 - val_loss: 0.3688
Epoch 2/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8839 - loss: 0.2776 - val_accuracy: 0.9352 - val_loss: 0.1784
Epoch 3/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8996 - loss: 0.2467 - val_accuracy: 0.9314 - val_loss: 0.1848
Epoch 4/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9101 - loss: 0.2210 - val_accuracy: 0.9733 - val_loss: 0.0789
Epoch 5/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9178 - loss: 0.2073 - val_accuracy: 0.7531 - val_loss: 0.5723
Epoch 6/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9243 - loss: 0.1914 - val_accuracy: 0.9255 - val_loss: 0.1932
Epoch 7/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9282 - loss: 0.1803 - val_accuracy: 0.9012 - val_loss: 0.2423
Epoch 8/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9323 - loss: 0

In [40]:
# Save and export the model
model.save("img_classifier.keras")

In [41]:
# Predict new images
model = models.load_model("./img_classifier.keras")

newImgs = []

for filename in os.listdir("./data"):
    imgPath = os.path.join("./data", filename)
    if os.path.isfile(imgPath):
        img = cv2.imread(imgPath)
        if img is not None:
            img = cv2.resize(img, (32, 32))
            newImgs.append(img)
            print(filename)

newImgs = np.array(newImgs)

predictions = model.predict(newImgs)

for i, pred in enumerate(predictions):
    print(f"Image {i+1} prediction: {pred[0]:.10f}")

fake.png
frog1.jpg
frog2.jpg
frog3.png
frog4.jpg
frog5.jpg
frog6.jpg
real.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Image 1 prediction: 0.0003023147
Image 2 prediction: 0.0000000217
Image 3 prediction: 0.0000016659
Image 4 prediction: 0.0000041612
Image 5 prediction: 0.0000000001
Image 6 prediction: 0.0000958420
Image 7 prediction: 0.0000008647
Image 8 prediction: 0.0000013811
